# Types in Julia

This notebook will give you a quick overview of the Julia type system and how to use it. Types answer the question: "What kind of thing is this?" 

In [ ]:
typeof(1)

In [ ]:
typeof(1.)

In [ ]:
typeof("hello")

All objects in Julia have an associated "type". 

## User Defined Types

`Int64`, `Float64` etc. are built-in Julia types. But user defined types in Julia are also treated as first class citizens. 

In [ ]:
struct Vol1
    value
end

In [ ]:
V = Vol1(3)

We can show this nicely by overloading the `show` function on our type. The `show` method controls how objects are displayed. Remember from our notebook on multiple dispatch that functions can be overloaded or "imbued" with new functionality. To modify how `Vol1` is displayed, it so happens that the method we must overload is `show`. 

In [ ]:
Base.show

But `show` is a function that's only available in the `Base` module. First, we must import it to our namespace and then overload it. 

In [ ]:
import Base: show

show(io::IO, V::Vol1) = print(io, "Volume with value ", V.value)

In [ ]:
V = Vol1(3)

We can define e.g. the sum of two volumes. To do this, we must import the `+` function from `Base` again.

**Caution**: Be careful when using functions like `+`. Redefining their behaviour for standard operations (like adding two integers) could have unintended side effects and could cause Julia to crash. 

Why does this have unintended side effects?

Because you're modifying the behaviour of a method (not just function) that a lot of other functions (and methods) depend on. 

In [ ]:
import Base: +

+(V1::Vol1, V2::Vol1) = Vol1(V1.value + V2.value)

In [ ]:
V + V

But the following does not work, since we haven't defined `*` yet on our type:

In [ ]:
2V

There is a problem with our definition:

In [ ]:
Vol1("hello")

It doesn't make sense to have a string as a volume. So we should **restrict** which kinds of `value` are allowed, i.e. the **type** of `value`:

In [ ]:
struct Vol2
    value::Float64
end

In [ ]:
Vol2(3.1)

In [ ]:
Vol2("hello")

## Different types of volume: **parameteric types**

Now we can imagine that in different contexts, we could want integer volumes, or rational volumes, rather than Vols which contain a floating-point number, e.g. for a 3D printer that makes everything out of cubes of the same size.

We could define the following sequence of different types.

In [ ]:
struct Vol_Int
    value::Int
end

struct Vol_Float
    value::Float64
end

struct Vol_Rational
    value::Rational{Int64}
end

In [ ]:
Vol_Int(3)

In [ ]:
Vol_Int(3.1)

In [ ]:
Vol_Float(3.1)

But clearly this is the wrong way to do it, since we're repeating ourselves, and there is a strong principle not to do so (https://en.wikipedia.org/wiki/Don't_repeat_yourself).

Isn't there a more efficient way, where Julia itself can generate all of these different types?

What we would like to do is tell Julia that the **type** itself (here, `Int`, `Float64` or `Rational{Int64}`) 
is a **parameter** that we will specify. 

The syntax in Julia for this is to use curly braces (`{`, `}`) to specify such a **type parameter**:

In [ ]:
struct Vol3{T}
    value
end

We can now pass in **any type** and `T` will be replaced by that type, creating a new type, e.g.

In [ ]:
V = Vol3{Float64}(3.1)

In [ ]:
typeof(V)

In [ ]:
V2 = Vol3{Int64}(4)

In [ ]:
typeof(V2)

The type `Vol3` is called a **parametric type**, with **type parameter** `T`. Parameteric types may have several type parameters, as we have already seen with `Array`s:

In [ ]:
a = [3, 4, 5]
typeof(a)

The type parameters here are `Int64`, which is itself a type, and the number `1`.

## Improving the solution

The problem with this solution is the following, which echos what happened at the start of the notebook:

In [ ]:
V = Vol3{Int64}(3.1)

In [ ]:
typeof(V.value)

The type of the element (here, `Float64`) is disconnected from the type parameter (`Int64`). 
So we have not yet actually captured the pattern of `Vol2`,
which restricted the `value` field to be of the desired type.

We solve this be specifying the field to **also be of type `T`**, with the **same `T`**:

In [ ]:
struct Vol4{T}
    value::T
end

For example,

In [ ]:
V = Vol4{Int64}(3.0)

In [ ]:
typeof(V.value)

Now when we try to do 

In [ ]:
Vol4{Int64}(3.1)

Julia throws an error, namely an `InexactError()`.
This means that we are trying to "force" the number 3.1 into a "smaller" type `Int64`, i.e. one in which it can't be represented.

However, now we seem to be repeating ourselves again: We know that `3.1` is of type `Float64`, and in fact Julia knows this too; so it seems redundant to have to specify it. Can't Julia just work it out? Indeed it can!:

In [ ]:
Vol2(3.1)

Here, Julia has **inferred** the type `T` from the "inside out". That is, it did some pattern matching to realise that `value::T` was **matched** if `T` was chosen to be `Float64`, and then propagated this same value of `T` **upwards** to the type parameter.

**Exercise**: Define a `Point` type that represents a point in 2D, with two fields. What are the options for this type, mirroring the types `Vol1` through `Vol4`?